In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from nltk.tokenize import WordPunctTokenizer

# Data base import

In [ ]:
train_data =  pd.read_csv(Path("../data/samples/sample_1000_train.csv")).to_numpy()
validation_data = pd.read_csv(Path("../data/samples/sample_100_validation.csv")).to_numpy()

In [ ]:
print(train_data.shape, validation_data.shape)

# Pre-processing

In [ ]:
from utils.load_data import load_data
from utils.data_descriptor import convert_labels

ALPHANUM_ONLY = False
WORD_SIZE = 30
SENTENCE_SIZE = 50
FILL_WITH = "$"
SPLIT_PUNCTUATION = False # to tell wheter the puncutation "!?.;,/" etc are keeped sticked to a word or not

X_TR_STRING, X_TR_SCALAR = load_data(
    train_data, WORD_SIZE, SENTENCE_SIZE, FILL_WITH, SPLIT_PUNCTUATION, ALPHANUM_ONLY
)
X_TR_ORIGINAL = train_data[:, 1]
Y_TR = convert_labels(train_data, SENTENCE_SIZE, SPLIT_PUNCTUATION)

X_VAL_STRING, X_VAL_SCALAR = load_data(
    validation_data, WORD_SIZE, SENTENCE_SIZE, FILL_WITH, SPLIT_PUNCTUATION, ALPHANUM_ONLY
)
X_VAL_ORIGINAL = validation_data[:, 1]
Y_VAL = convert_labels(validation_data, SENTENCE_SIZE, SPLIT_PUNCTUATION)

In [ ]:
print(train_data.shape, X_TR_STRING.shape, X_TR_SCALAR.shape, Y_TR.shape)

print("\nOriginal data :")
print(X_TR_ORIGINAL[0])
print("\nFilled sentence :")
print(X_TR_STRING[0])
print("\nLabel :")
print(Y_TR[0])
print("\nDescriptor :")
print(X_TR_SCALAR[0])

# Classification

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

In [ ]:
nb_neighbors = 1

knn = KNeighborsRegressor(nb_neighbors, weights="distance")
knn.fit(X_TR_SCALAR, Y_TR)

In [ ]:
predictions = knn.predict(X_VAL_SCALAR)

In [ ]:
from utils.post_processing import pred_to_string

results = np.zeros(len(predictions), dtype=object)
for i in range(len(predictions)):
    results[i] = pred_to_string(X_TR_ORIGINAL[i], X_TR_STRING[i], predictions[i])

In [ ]:
for i in range(len(results)):
    print(DATA[i][1])
    print(DATA[i][2])
    print(results[i])
    print()

In [ ]:
from utils.loss import jaccard

avg = 0
for i in range(len(results)):
    avg += jaccard(results[i], DATA[i, 2])
    
    if jaccard(results[i], DATA[i, 2]) != 1:
        print("Error on", i)
avg /= len(results)

print(avg)